### Loop through csv files subfolder and create delta tables

In [ ]:
# Obtain list of files
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
import os
file_path = f"/lakehouse/default/Files/csv_files"
lst = os.listdir(file_path)
lst

In [ ]:
# Function to clean dataframe prior to saving as Delta tables
def clean_data(df):
    for col_name in df.columns:
        df = df.withColumnRenamed(col_name, col_name.replace(" ", "_"))
    # Check if the 'Unit_Price' column exists and cast it to DoubleType
    if 'Unit_Price' in df.columns:
        df = df.withColumn('Unit_Price', col('Unit_Price').cast(DoubleType()))
    return df


for file in lst:
    if file.endswith('.csv'):
        source_file_path = os.path.join(qualified_file_path, file)

        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load(source_file_path)

        df_clean = clean_data(df)

        file_name_without_extension = os.path.basename(
            source_file_path).split('.')[0]
        file_name_without_extension = file_name_without_extension.replace(
            " ", "_").replace("(", "_").replace(")", "_")
        delta_table_path = os.path.join(
            delta_qualified_path, file_name_without_extension)

        df_clean.write.format("delta") \
            .mode("overwrite") \
            .option("mergeSchema", "true") \
            .save(delta_table_path)